## Aggregating Data & Running Multivariate Clustering
This notebook is a walk-through of the following steps:
- Aggregating data using two different approaches: (1) binning the data into hexagon bins, (2) using aggregated point-level data
- Running the multivariate clustering using the k-medoids method
- Visualizing the clustering output onto a map
- Interpreting the clustering results

#### There are several ways to aggregate data before running the multivariate clustering
1. One option is to aggregate values from each parameter into hexagonal bins of a defined size, and then run the clustering
2. Another option is to down-sample one of the bathymetry layers, generate a feature class of point data with all of the other layers, run the clustering, then convert the point feature class to a new raster

### <u>First Option: Aggregate data using hexagonal bins<u>

1. Convert one of the raster layers to points using the **Raster to Point** tool. This step will generate a new [feature class](https://pro.arcgis.com/en/pro-app/latest/help/data/feature-classes/feature-classes.htm).

2. Add other layers (BTM output and environmental layers) to the feature class using the **Extract Multi Values to Points** tool. After this step, you should have an attribute table for this layer containing all of the variables. Note that in this case, the temperature, salinity, and POC flux all have redundant values. This is because the bathymetry layers are higher resolution than the environmental layers (this will not be the case after the next step is run).<br><br>
<img src="images/rastertopoint_table.png" style="width:600px; height:auto;" />

3. Use the **Summarize Within** GeoAnalytics Desktop Tool to generate hexagonal bins of the data. <br>
- First, set the Summarized Layer as the feature class generated in the previous steps, select a "Bin" under "Polygon or Bin", and "Hexagon" as the Bin Type. From there, you can specify the size of the bin that you want. <br>
- BBPI and FBPI values are categorical, so it doesn't make sense to calculate a mean of all BBPI/FBPI values in a 5km bin. Instead, we can allocate "Any" BBPI/FBPI value within the bin, which will randomly select a value and assign it to a particular bin.<br>
- Once you've run this step, you will get an output feature class containing all of the binned data. You can visualize any of these fields by right-clicking on the layer in the Contents Pane and selecting "Symbology". <br>

<img src="images/sal_bin.png" style="width:600px; height:auto;" /> <img src="images/sal_bin_legend.png" style="width:150px; height:auto;" />

#### Second Option: Aggregate raster and convert to points

1. **Change raster resolution**: The GEBCO bathymetry data is at ~450m resolution, while the Bio-ORACLE and POC flux layers are much coarser (1/12-degree resolution or ~9km). To have grid cells of adequate size for all layers, we will down-sample one of the bathymetry layers (in this case to 5km resolution) using the **Aggregate** Spatial Analyst tool, using a cell factor of 11. Any of the bathymetry layers can be used here (bathymetry, BBPI, FBPI, or slope). <br>**Note**: If either the BBPI or FBPI layers are used in this step, it is important to use the median aggregation technique (and not the mean) since the BBPI/FBPI values are categorical. If the slope layer is used, the mean slope value can be used.<br>

2. Convert the aggregated raster from the previous step to points using the **Raster to Points** tool. 

3. Use the **Extract Multi Values to Points** tool to add other layers to the feature class generated in the previous steps. Select the input rasters to include in the feature class: all BTM output layers, Bio-ORACLE layers, and benthic POC flux model layer.

#### Running the Multivariate Clustering 

Whether you used the first or second option to aggregate your data, the clustering steps are the same. <br>Use the **[Multivariate Clustering](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/multivariate-clustering.htm)** Spatial Statistics tool by doing the following:
- Input Features: select the feature class generated in the previous steps (either the hex-binned feature class, or the aggregated point feature class) 
- Select the Analysis Fields to be used in the clustering (temperature, salinity, poc flux, bbpi, fbpi, slope). 
- Select the k-medoids clustering method, and the "optimized seed locatios" as the initialization method
- Enter a name/path for an output table for evaluating the number of clusters <br>

<img src="images/clustering_settings.png" style="width:400px; height:auto;" />

Once the clustering is done running, you can open the Pseudo-F statistic chart to evaluate the number of clusters that you want to use for your analysis. The Pseudo-F statistic describes the ratio of between cluster variance to within-cluster variance. Once you've decided the number of clusters to use, re-run the clustering, this time inputting a number under "Number of Clusters".

<img src="images/pseudo_f_allparams.png" style="width:700px; height:auto;" />

#### Visualizing clustering output on a map

Using hex-bins, the clustering output can be visualized directly onto a map:

<img src="images/clustering_bins.png" style="width:700px; height:auto;" />

Using the point feature class output, the layer can be converted to a raster by using the **Point to Raster** tool. The raster can then be visualized onto a map: 

<img src="images/clustering_raster.png" style="width:400px; height:auto;" />

#### Interpreting clustering outputs

As a first step, we can look at the distribution of cell counts corresponding to the different cluster IDs. This figure is automatically generated during the multivariate clustering step and can be found in the Contents Pane. In this example, (which is the output from the U.S West Coast-wide point feature class clustering), we can see that the cluster ID #1 is has the highest number of point counts associated with it. As we can see from the map view of the output, this corresponds to offshore water which likely has a relatively flat terrain, and cold, high-salinity water. Meanwhile, there are very few points associated with Cluster ID #7, potentially meaning that this Cluster ID is associated with a rare habitat type.

<img src="images/counts.png" style="width:700px; height:auto;" />

We can also use box-plots to show the distribution of different variables' standardized values (z-scores) split by Cluster ID. This figure is automatically generated during the multivariate clustering step and can be found in the Contents Pane. 
- On the left panel below: values corresponding to the cluster "4" (dark green) tends to be productive (high POC flux), low salinity, and higher temperature. Looking back at the raster map of the cluster IDs, those cells tend to be coastal, as expected. 
- On the right panel below: values corresponding to the cluster "2" (dark blue) tends to have a higher fine scale BPI value and a higher slope, meaning these cells are likely associated with steep, small-scale benthic features.

<img src="images/boxplot1.png" style="width:700px; height:auto;" /> <img src="images/boxplot2.png" style="width:685px; height:auto;" />

You can also visualize the distribution of a particular variable as a function of cluster ID. From the content pane, right click on the clustering output layer, then select "Create Chart" and select "Box Plot". Under "Numerical Fields", add the variable that you want to plot, and under "Category", select "Cluster ID". Below are examples of those plots for temperature and salinity.

<img src="images/temp_boxplot_2.png" style="width:700px; height:auto;" /> <img src="images/sal_boxplot.png" style="width:700px; height:auto;" />

Another useful tool is to generate a scatter plot between two variables, split by Cluster ID (see example below). For more exploratory analysis, a scatter plot matrix can also be generated.

<img src="images/ts_scatter.png" style="width:600px; height:auto;" />

#### Optional: integrating other datasets (e.g., shapefiles)

Adding other point feature class data or shapefiles as a layer on a map if very easy. [Here is a link](https://pro.arcgis.com/en/pro-app/latest/help/data/shapefiles/working-with-shapefiles-in-arcgis-pro.htm) with information about adding shapefiles into ArcGIS Pro. <br>For example, below is a shapefile of the Monterey Bay National Marine Sanctuary as a layer on top of the clustering output. 

<img src="images/mb_shp.png" style="width:600px; height:auto;" />